In [ ]:
# Imports
import scipy.io.wavfile
import scipy.fftpack
import scipy.signal
import numpy as np

In [ ]:
# Graph Imports
import cufflinks as cf
import plotly.express as px
import plotly.offline
import plotly.graph_objects as go
import psutil

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
# Read Wav
def readMonoWAV(filename):
  wavFileSampleRate, wavFileSamplesLR = scipy.io.wavfile.read('lick.wav')
  wavFileSamplesLR = [((x[0] + x[1]) / 2) for x in wavFileSamplesLR]
  return (wavFileSampleRate, wavFileSamplesLR)

wavFileSampleRate, wavFileSamplesLR = readMonoWAV('lick.wav')

In [ ]:
# Show WAV
waveformFig = px.scatter(y=wavFileSamplesLR, x=range(len(wavFileSamplesLR)))
waveformFig.show()

In [ ]:
def findFFT(samples,sampleRate,activationLevel):
    numSamples = len(wavFileSamplesLR)
    timeStep = (1 /sampleRate)
    
    chunk = samples
    chunkFFT = np.abs(scipy.fftpack.fft(chunk))
    chunkFFTFreqs = scipy.fftpack.fftfreq(int(len(chunkFFT)),d=timeStep)
    chunkFFTPos = chunkFFT[:int(abs(len(chunkFFT)))]

    chunkVolumeArr = np.array(samples)
    chunkVolumeArr = np.absolute(chunkVolumeArr)
    chunkVolume = np.mean(chunkVolumeArr)

    return (chunkFFT,chunkFFTPos, chunkFFTFreqs, chunkVolume)

def getDominantFreqs(samples,sampleRate,chunkSize=0, activationLevel=12.5):
    if chunkSize==0:
        chunkSize = int(sampleRate / 20 )
    startChunk = 0
    endChunk = startChunk + chunkSize
    activationLevel = max(samples) * (activationLevel * 0.01)
    print(chunkSize)
    numSamples = len(samples)
    print(numSamples)
    domFreqs = []
    while (endChunk < numSamples):
        (chunkFFT,chunkFFTPos, chunkFFTFreqs, chunkVolume) = findFFT(samples[startChunk:endChunk],sampleRate,activationLevel)
        if (chunkVolume < activationLevel):
            startChunk = endChunk
            endChunk += chunkSize
            domFreqs.append(0.0)
            continue

        FFTPeaksIdx, _ = scipy.signal.find_peaks(chunkFFTPos)
        FFTPeaks = [chunkFFTFreqs[x] for x in FFTPeaksIdx]
        domFreq = chunkFFTFreqs[np.argmax(chunkFFT)]
        domFreqs.append(domFreq)
        print(domFreq)
        startChunk = endChunk
        endChunk += chunkSize

    timeWindow = chunkSize / sampleRate
    return (domFreqs, timeWindow)

(domFreqs, timeWindow) = getDominantFreqs(wavFileSamplesLR, wavFileSampleRate,chunkSize=0, activationLevel=12.5)







In [7]:
print((domFreqs, timeWindow))

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 0.0, 0.0, 340.0, 340.0, 340.0, 340.0, 0.0, 0.0, 0.0, 1400.0, 1400.0, 1400.0, 360.0, 0.0, 0.0, 0.0, 400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 0.0, 0.0, 0.0, 340.0, 340.0, 340.0, 340.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 260.0, 260.0, 260.0, 0.0, 0.0, 0.0, 0.0, 0.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [10]:
def generateSinFromFreqs(domFreqs, timeWindow, outputSampleRate, fileOutput):
    finalSin = []
    sinArr = []

    for frequency in domFreqs:
        t = np.linspace(0, timeWindow, outputSampleRate * timeWindow )  #  Produces a 5 second Audio-File
        y = np.sin(frequency * 2 * np.pi * t) 
        sinArr.append(y)

    finalSin = np.concatenate(sinArr)

    if not fileOutput.endswith('.wav'):
        fileOutput += '.wav'

    scipy.io.wavfile.write(fileOutput, outputSampleRate, finalSin)

    print(f'File Successfully written to "{fileOutput}" at Sample Rate: {outputSampleRate}')


generateSinFromFreqs(domFreqs, timeWindow, outputSampleRate=48000, fileOutput='LickSinWave')

File Successfully written to "LickSinWave.wav" at Sample Rate: 48000


0

In [ ]:

length = (chunkSize / wavFileSampleRate )

finalSin = []
sinArr = []
for frequency in domFreqs:
    t = np.linspace(0,length, wavFileSampleRate * length)  #  Produces a 5 second Audio-File
    y = np.sin(frequency * 2 * np.pi * t)  #  Has frequency of 440Hz
    sinArr.append(y)

finalSin = np.concatenate(sinArr)
scipy.io.wavfile.write('lick_Sin.wav', wavFileSampleRate, finalSin)

In [12]:
from mido import *


def writeMIDI(domFreqs, timeWindow):
    def convertToMIDINote():
        
    
    
    mid = mido.MidiFile()
    track = mido.MidiTrack()

    prevFrequency = 0

    for frequency in domFreqs:
        if frequency == 0:
            prevFrequency = frequency 
            continue
        if frequency != prevFrequency:
            track.append(mido.Message('note_on', ))



In [14]:
a = 440
freqs = []
count = 1
count
while (count < 7):
    a = a / ((2 ** (1 / 12)) ** count )
    freqs.append(a)
    count += 1

print(freqs)



[415.3046975799451, 369.9944227116344, 311.12698372208087, 246.94165062806198, 184.99721135581711, 130.81278265029923]
